[LANGCHAIN で日本語プロンプトを扱う (REASONING)](https://netweblog.wordpress.com/2023/04/03/langchain-japanese/)

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "http://localhost:8000"

In [7]:
model_name="text-davinci-003"

import re
from typing import Sequence, List, Tuple, Optional, Any, Union
from langchain.agents.agent import Agent, AgentOutputParser
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.base import BasePromptTemplate
from langchain.tools.base import BaseTool
from langchain.agents import Tool, initialize_agent, AgentExecutor
from langchain.llms import OpenAI
from langchain.schema import AgentAction, AgentFinish, OutputParserException

##########
# define tools
##########

company_dic = {
  "A": 2000,
  "B": 1500,
  "C": 20000,
  "D": 6700,
  "E": 1000,
  "F": 4100,
}

def get_invoice(company_name):
  return company_dic[company_name]

def diff(value_str):
  str_list = value_str.split(" ")
  assert(len(str_list) == 2)
  int_list = [int(s) for s in str_list]
  return str(abs(int_list[0] - int_list[1]))

def total(value_str):
  str_list = value_str.split(" ")
  int_list = [int(s) for s in str_list]
  return str(sum(int_list))

tools = [
  Tool(
    name="GetInvoice",
    func=get_invoice,
    description="Get invoice amount of trading company.",
  ),
  Tool(
    name="Diff",
    func=diff,
    description="Get diffrence.",
  ),
  Tool(
    name="Total",
    func=total,
    description="Get total.",
  ),
]

##########
# define agent
##########

EXAMPLES = [
  """質問 : 会社 A の請求金額と会社 B の請求金額との間の差額はいくらか?
考察 : 会社 A の請求金額を取得する必要がある。
行動 : GetInvoice[A]
結果 : 2000
考察 : 会社 B の請求金額を取得する必要がある。
行動 : GetInvoice[B]
結果 : 1500
考察 : 会社 A の請求金額と会社 B の請求金額との間の差額を計算する必要がある。
行動 : Diff[2000 1500]
結果 : 500
考察 : よって、答えは 500。
行動 : Finish[500]""",
  """質問 : 会社 B, C, D の請求金額の合計はいくらか?
考察 : 会社 B の請求金額を取得する必要がある。
行動 : GetInvoice[B]
結果 : 1500
考察 : 会社 C の請求金額を取得する必要がある。
行動 : GetInvoice[C]
結果 : 20000
考察 : 会社 D の請求金額を取得する必要がある。
行動 : GetInvoice[D]
結果 : 6700
考察 : 会社 B, C, D の請求金額の合計を計算する必要がある。
行動 : Total[1500 20000 6700]
結果 : 28200
考察 : よって、答えは 28200。
行動 : Finish[28200]""",
  """質問 : 会社 C の請求金額と会社 A, D の請求金額の合計との間の差額はいくらか?
考察 : 会社 C の請求金額を取得する必要がある。
行動 : GetInvoice[C]
結果 : 20000
考察 : 会社 A の請求金額を取得する必要がある。
行動 : GetInvoice[A]
結果 : 2000
考察 : 会社 D の請求金額を取得する必要がある。
行動 : GetInvoice[D]
結果 : 6700
考察 : 会社 A, D の請求金額の合計を計算する必要がある。
行動 : Total[2000 6700]
結果 : 8700
考察 : 会社 C の請求金額と会社 A, D の請求金額の合計との間の差額を計算する必要がある。
行動 : Diff[20000 8700]
結果 : 11300
考察 : よって、答えは 11300。
行動 : Finish[11300]""",
]

SUFFIX = """\n質問 : {input}
{agent_scratchpad}"""

TEST_PROMPT = PromptTemplate.from_examples(
  EXAMPLES, SUFFIX, ["input", "agent_scratchpad"]
)


# Prompt Exampleに合わせてテキスト処理
# Action句を解析して、SearchなどのActionに移るか、終わらせるかを判断する
class ReActOutputParser(AgentOutputParser):
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        action_prefix = "行動 : "
        if not text.strip().split("\n")[-1].startswith(action_prefix):
            raise OutputParserException(f"Could not parse LLM Output: {text}")
        action_block = text.strip().split("\n")[-1]

        action_str = action_block[len(action_prefix) :]

        re_matches = re.search(r"(.*?)\[(.*?)\]", action_str)
        if re_matches is None:
            raise OutputParserException(
                f"Could not parse action directive: {action_str}"
            )
        action, action_input = re_matches.group(1), re_matches.group(2)

        # 最後が行動: Finishであれば処理を終わらせる
        if action == "Finish":
            return AgentFinish({"output": action_input}, text)
        else:
            return AgentAction(action, action_input, text)


class ReActTestAgent(Agent):

  # Action句を解析して、SearchなどのActionに移るか、終わらせるかを判断する
  @classmethod
  def _get_default_output_parser(cls, **kwargs: Any) -> AgentOutputParser:
    return ReActOutputParser()

  @classmethod
  def create_prompt(cls, tools: Sequence[BaseTool]) -> BasePromptTemplate:
    return TEST_PROMPT

  @classmethod
  def _validate_tools(cls, tools: Sequence[BaseTool]) -> None:
    if len(tools) != 3:
      raise ValueError("エラー : ツールの数が正しくありません")
    tool_names = {tool.name for tool in tools}
    if tool_names != {"GetInvoice", "Diff", "Total"}:
      raise ValueError("エラー : ツールの名前が正しくありません")

  @property
  def _agent_type(self) -> str:
    return "react-test"

  @property
  def finish_tool_name(self) -> str:
    return "Finish"

  @property
  def observation_prefix(self) -> str:
    return f"結果 : "

  @property
  def _stop(self) -> List[str]:
   return [f"\n結果 : "]

  @property
  def llm_prefix(self) -> str:
    return f"考察 : "


##########
# run agent
##########

llm = OpenAI(
  model_name=model_name,
  temperature=0,
)
agent = ReActTestAgent.from_llm_and_tools(
  llm,
  tools,
)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent,
  tools=tools,
  verbose=True,
)

question = "会社 C, F の請求金額の合計と会社 A, E の請求金額の合計との間の差額はいくらか?"
agent_executor.run(question)



> Entering new AgentExecutor chain...
考察 : 会社 C の請求金額を取得する必要がある。
行動 : GetInvoice[C]
結果 : 20000
考察 : 会社 F の請求金額を取得する必要がある。
行動 : GetInvoice[F]
結果 : 4100
考察 : 会社 C, F の請求金額の合計を計算する必要がある。
行動 : Total[20000 4100]
結果 : 24100
考察 : 会社 A の請求金額を取得する必要がある。
行動 : GetInvoice[A]
結果 : 2000
考察 : 会社 E の請求金額を取得する必要がある。
行動 : GetInvoice[E]
結果 : 1000
考察 : 会社 A, E の請求金額の合計を計算する必要がある。
行動 : Total[2000 1000]
結果 : 3000
考察 : 会社 C, F の請求金額の合計と会社 A, E の請求金額の合計との間の差額を計算する必要がある。
行動 : Diff[24100 3000]
結果 : 21100
考察 :  よって、答えは 21100。
行動 : Finish[21100]

> Finished chain.


'21100'